### Lab:3 - 02.Comparing base composition

### Programming: Composition-based distances
Computing phylogenies for species based on whole genomes is hard for several reasons. Do you work with genes only? How do you then pick the genes? You cannot, in general, align whole genomes because of recombination events that cause large scale differences.

One solution that has been tried is comparing the difference in nucleotide composition. Let πX be the nucleotide composition vector for species X. If all nucleotides have the same frequence in X (which would be surprising), then πX=(0.25, 0.25, 0.25, 0.25).

If genome Y we are looking at is GC rich, then the composition vector might be πY = (0.2, 0.3, 0.3, 0.2), assuming that the nucleotide frequencies are in the order A, C, G, and T.

 

### Measuring difference in composition
To be able to use composition for distances, we want to have a measure for the difference. In this assignment, we use the root-mean-square distance:

Take the elementwise differences.
Square the differences and add them up.
Divide the sum by 4 and take the square root.
diff(πX,πY) = √ (0.25 * ((0.25-0.2)2 + (0.25-0.3)2 + (0.25-0.3)2 + (0.25-0.2)2))

### Assignment
Write a Python program that takes as input a number of filenames, each pointing to files containing one genomic sequence in Fasta format, and output a distance matrix using the composition difference above.

Suggested solution structure
Define a function dist that takes two composition vectors as arguments and returns a difference.

In [1]:
import urllib.request
import math
import numpy
import sys

In [2]:
def readGenome(filename):
    """
    readGenome is a function that opens a Fasta format file available in the wd and reads the sequences on it.
    :param filename: name of the file on the wd.
    :return: a dictionary with genome name as key and the sequence as keyvalue.
    """
    # dna_sequences_by_name_dict is a dictionary with keys: genome name, values: sequence.
    dna_sequences_by_name_dict = dict()
    # Opens the file listed in filenames.
    with open(filename, 'r') as f:
        # for each line on the file:
        for line in f:
            # removes the entry key.
            line = line.strip()
            # if the line starts with > the key value will be empty, while the key will take the value of the line.
            if line[0] == '>':
                genome = ''
                sequence_name = line
            # if the line do not starts with > the keyvalue will take the line value and will join all of the next strings until next >.
            if line[0] != '>':
                genome += line
            dna_sequences_by_name_dict[sequence_name] = genome
    # returns the dictionary with all the read sequences of the file.
    return dna_sequences_by_name_dict

In [3]:
#for sequence_name, dna_sequence in translate_seq.items():
def composition_vector(dna_sequence):
    """
    composition_vector is a function that weights the nucleotides (A,C,G,T) frequencies on the dna_sequence.
    :param dna_sequence: DNA sequence 
    :return: vector with three elements as percentages of the composition weight (A,C,G,T).
    """
    content_A = dna_sequence.count('A')
    content_C = dna_sequence.count('C')
    content_G = dna_sequence.count('G')
    content_T = dna_sequence.count('T')
    length = float(content_A + content_C + content_G + content_T)
    length_other_nucleotides = len(dna_sequence) - length
    
    if length_other_nucleotides != 0:
        percentage_other_nucleotides= round(float(length_other_nucleotides/len(dna_sequence)),4)
        
        print(
            f'WARNING: The genome sequence contains {percentage_other_nucleotides * 100} %'
            ' of nucleotides other than A,C,G,T ',
            file=sys.stderr
        )
    return (
        round(float(content_A / length), 4),
        round(float(content_C / length), 4),
        round(float(content_G / length), 4),
        round(float(content_T / length), 4)
    )

In [4]:
def distance(a_comp_vector, b_comp_vector):
    """
    distance is a function that measueres the distance of two vectors, using the root-mean-square distance.
    :param a_comp_vector: compasition vector for a given genome a.
    :param b_comp_vector: compasition vector for a given genome b.
    :return: single value with the two vectors distance.
    """
    d = [
        ((a_comp_vector[0] - b_comp_vector[0])**2), ((a_comp_vector[1] - b_comp_vector[1])**2), 
         ((a_comp_vector[2] - b_comp_vector[2])**2)
        ]
    
    return round(math.sqrt(0.25*(d[0] + d[1] + d[2])), 4)

In [5]:
def format_to_ten_characters(sequence_name):
    """
    format_to_ten_characters is a function that limits the sequence_name length to 10.
    :param sequence_name: 
    :return: sequence_name limited to 10 characters.
    
    """
    # if the sequence length is greater than 10 take just the first 10 characters and add 4 spaces this to create 
    # more distance on the matrix.
    if len(sequence_name) > 10:
        return sequence_name[:10] + '  '
    # if it's less than ten add blank spaces for the remaining, plus 4 more spaces.
    return sequence_name + ' ' * (10 - len(sequence_name)) + '  '


In [6]:
def checking_for_errors_in_filename(filenames_list):
    """
    checking_for_errors_in_filename is a function that evaluates two scenarios in which the file reading could go wrong.
    :param filenames_list: is a list containing filenames passed as arguments from the command line.
    :return:
    """

    for filename in filenames_list:
        try:
            with open(filename, 'r') as my_file:
                pass
        except Exception as e:
            return True, e
        # if the filename extension is not 'fa' raise an error message.
        if filename[-3:] != '.fa':
            return True, 'only fasta format accepted'

    return False, ''

In [24]:
def convert_matrix_row(matrix_row):
    """
    matrix_row is a function that edits the matrix rows so no brackets are written on the output file
    :param matrix_row:
    :return:
    """
    matrix_str_row = '_'
    for value in matrix_row:
        matrix_str_row += f'{value}'+ ' '
    return matrix_str_row[:-1] + '\n'

In [26]:
convert_matrix_row(my_little_matrix)

'['

In [13]:
## Download and write files: each file contains parts of the chromosomes/genomes for each species.

urls =["http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/simple1.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/simple2.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/human.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/mouse.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/fly.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/yeast.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/ecoli.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/plasmodium.fa",
    "http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/thermus.fa"]

filenames = []
for gene in urls:
    filenames.append(gene.split('lab2/')[1])
##
url_by_genome_name = dict(zip(filenames, urls))

In [27]:
url_by_genome_name

{'simple1.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/simple1.fa',
 'simple2.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/simple2.fa',
 'human.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/human.fa',
 'mouse.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/mouse.fa',
 'fly.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/fly.fa',
 'yeast.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/yeast.fa',
 'ecoli.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/ecoli.fa',
 'plasmodium.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/plasmodium.fa',
 'thermus.fa': 'http://www.csc.kth.se/utbildning/kth/kurser/DD2404/appbio12/labs/lab2/thermus.fa'}

In [14]:
# for key, keyvalue in the dictionary items.
for filename, url in url_by_genome_name.items():
    with urllib.request.urlopen(url) as url:
        gene = url.read()
        gene = gene.strip().decode("utf-8")
    with open(filename, 'w') as f_out:
        f_out.write(gene)
       

In [16]:
 for filename in filenames:
     read_gene =readGenome('simple1.fa')
     read_gene2 = readGenome('simple2.fa')
filenames_list = ['simple1.fa', 'simple2.fa', 'human.fa', 'mouse.fa']

In [17]:
dna_sequence_by_name = dict()
for filename in filenames_list:
    translate_seq = readGenome(filename)
    dna_sequence_by_name = {**dna_sequence_by_name, **translate_seq}
composition_dna_sequence_by_name = {
    sequ_name[:10]: composition_vector(dna_sequence) for sequ_name, dna_sequence in dna_sequence_by_name.items()
}

In [18]:
composition_dna_sequence_by_name

{'>simple1': (0.25, 0.25, 0.25, 0.25),
 '>simple2': (0.3, 0.2, 0.2, 0.3),
 '>Hsapiens ': (0.2823, 0.2286, 0.2094, 0.2797),
 '>Mus_muscu': (0.284, 0.2172, 0.2225, 0.2763)}

In [19]:
my_little_matrix = numpy.zeros((len(composition_dna_sequence_by_name), len(composition_dna_sequence_by_name)))
for index, sequence_vector_a in enumerate(composition_dna_sequence_by_name.values()):
    for index_2, sequence_vector_b in enumerate(composition_dna_sequence_by_name.values()):
        my_little_matrix[index,index_2] = distance(sequence_vector_a,sequence_vector_b)
for index, sequence_name in enumerate(composition_dna_sequence_by_name.keys()):
    print(sequence_name, my_little_matrix[index])
        

    

>simple1 [0.     0.0433 0.0281 0.0273]
>simple2 [0.0433 0.     0.0175 0.0163]
>Hsapiens  [0.0281 0.0175 0.     0.0087]
>Mus_muscu [0.0273 0.0163 0.0087 0.    ]


In [44]:
if __name__ == '__main__':
    # filenames_list = list of entered arguments from command line.
    filenames_list = sys.argv[1:]
    dna_sequence_by_name = dict()
    for filename in filenames_list:
        translate_seq = readGenome(filename)
        dna_sequence_by_name = {**dna_sequence_by_name, **translate_seq}
    composition_dna_sequence_by_name = {
        sequ_name[:10]: composition_vector(dna_sequence) for sequ_name, dna_sequence in dna_sequence_by_name.items()
    }
    

[]


In [21]:
my_little_matrix

array([[0.    , 0.0433, 0.0281, 0.0273],
       [0.0433, 0.    , 0.0175, 0.0163],
       [0.0281, 0.0175, 0.    , 0.0087],
       [0.0273, 0.0163, 0.0087, 0.    ]])

In [64]:
new_dna_sequence_by_name = {seq_name: value**2 for seq_name, value in dna_sequence_by_name.items()}
new_dna_sequence_by_name

{'my_little_baby_0': 0, 'my_little_baby_1': 1, 'my_little_baby_2': 4}

In [61]:
dna_sequence_by_name

{'my_little_baby_0': 0, 'my_little_baby_1': 1, 'my_little_baby_2': 2}

In [85]:
read_gene['>simple1']

'ACGTACGTACGTACGTACGTACGTACGTACGTNNNNNNNNNNNNNNNNACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGTACGT'

In [90]:
composition_vector(read_gene['>simple1'])

(0.25, 0.25, 0.25, 0.25)

In [120]:
dna_sequence_by_name = dict()
for i in range(3):
    print('check our dna sequence dict before adding new one')
    print(dna_sequence_by_name)
    
    new_dict = {f'my_little_baby_{i}': i}
    dna_sequence_by_name = {**new_dict,**dna_sequence_by_name }
    
    print('check our dna sequence dict after adding new one')
    print(dna_sequence_by_name)
    print(' ')

check our dna sequence dict before adding new one
{}
check our dna sequence dict after adding new one
{'my_little_baby_0': 0}
 
check our dna sequence dict before adding new one
{'my_little_baby_0': 0}
check our dna sequence dict after adding new one
{'my_little_baby_1': 1, 'my_little_baby_0': 0}
 
check our dna sequence dict before adding new one
{'my_little_baby_1': 1, 'my_little_baby_0': 0}
check our dna sequence dict after adding new one
{'my_little_baby_2': 2, 'my_little_baby_1': 1, 'my_little_baby_0': 0}
 
